In [1]:
# 우리 도메인의 특성을 고려하여 필터링 과정을 RDB에 위임한 상황이다.
# 따라서 사용자 질문에서 키워드를 추출하고, 이를 SQL로 변환하여 동적 쿼리를 생성해야 한다.
# 방향성은 크게 두 가지인 것 같다.
# 1. 사용자 질의에서 데이터 추출(json 등)은 LLM이 하고, SQL 쿼리 변환은 코드가 담당하는 것,.
# 2. 데이터 추출부터 SQL 쿼리 변환까지 모두 LLM이 담당하는 것.
# 둘 중 무엇을 할까 하다가 1번을 하기로 하였다.
# 1번의 경우 유연성은 다소 떨어질 수 있지만, 안정성과 보안이 뛰어나며, 예측 및 제어 가능하다는 장점이 있기 때문이다.
# 2번은 반대로 유언성이 높지만 SQL injection 등 보안 공격에 취약하며 제어나 디버깅이 어렵다는 단점이 있기 때문이다.

# 가보자.
# 1. 사용자 질문이 들어온다 
# 2. LLM이 키워드를 추출한다(대신 이전에 비해 필터 수가 상당히 감소함. 나이, 거주지, 현재 신청 가능여부 정도만 필터링함). 
# 3. 이를 기반으로 코드 단에서 동적으로 SQL 쿼리를 생성하여 조건에 부함하는 정책 id를 가져온다.


In [1]:
# 1. 파서 함수 : 사용자의 질문을 받아 JSON 스키마에 맞춰 필터를 생성하는 함수.

import os
import json
from openai import OpenAI

def create_filter_from_query(client: OpenAI, user_query: str) -> dict:
    """
    사용자의 자연어 질문을 분석하고,
    정책 필터링에 사용할 구조화된 JSON(파이썬 딕셔너리)을 생성합니다.

    Args:
        client: 초기화된 OpenAI 클라이언트 객체
        user_query: 사용자의 원본 질문 문자열

    Returns:
        추출된 필터 정보가 담긴 딕셔너리
    """
    # LLM에게 전달할 시스템 프롬프트. 역할, 지시사항, 허용 값, JSON 스키마를 명시합니다.
    system_prompt = """
# ROLE
You are an expert at extracting key information for filtering South Korean youth policies from a user's query.

# INSTRUCTION
- Analyze the user's query and generate a JSON object that strictly follows the provided `JSON SCHEMA`.
- CRITICAL: When extracting regions, you MUST normalize them to their full official administrative names. (e.g., "서울", "서울시" -> "서울특별시" / "경기" -> "경기도" / "부산" -> "부산광역시" / "성남" -> "성남시" / "종로" -> "종로구")
- For fields with `ALLOWED VALUES`, you MUST choose from the provided list. If a user's term is a synonym, map it to the correct value (e.g., "실업자" -> "미취업").
- If a value is not mentioned, use `null` for single values or an empty list `[]` for array values.
- Do NOT make up values that are not in the `ALLOWED VALUES` list.
- Output ONLY the JSON object.

# ALLOWED VALUES
- 'job_status': ["재직자", "자영업자", "미취업자", "프리랜서", "일용근로자",
"(예비)창업자", "단기근로자", "영농종사자", "기타", "제한없음"]
- 'marriage_status': ["기혼", "미혼", "제한없음"]
- 'education_levels': ["고졸 미만", "고교 재학", "고졸 예정", "고교 졸업", "대학 재학", "대졸 예정",
"대학 졸업", "석·박사", "기타", "제한없음"]
- 'majors': ["인문계열", "사회계열", "상경계열", "이학계열", "공학계열", 
"예체능계열", "농산업계열", "기타", "제한없음"]
- 'categories': ["일자리", "주거", "교육", "복지문화", "참여권리"]
- 'subcategories': ["취업", "재직자", "창업", "주택 및 거주지", "기숙사",
"전월세 및 주거급여 지원", "미래역량강화", "교육비지원", "온라인교육", "취약계층 및 금융지원",
"건강", "예술인지원", "문화활동", "청년참여", "정책인프라구축", "청년국제교류", "권익보호"]
- 'specializations': ["중소기업", "여성", "기초생활수급자", "한부모가정", "장애인",
"농업인", "군인", "지역인재", "기타", "제한없음"]
- 'keywords': ["대출", "보조금", "바우처", "금리혜택", "교육지원", "맞춤형상담서비스",
"인턴", "벤처", "중소기업", "청년가장", "장기미취업청년", "공공임대주택",
"신용회복", "육아", "출산", "해외진출", "주거지원"]


# JSON SCHEMA
{
  "age": "number | null",
  "income": "number | null",
  "regions": ["string"],
  "job_status": ["string"],
  "marriage_status": "string | null",
  "education_levels": ["string"],
  "majors": ["string"],
  "categories": ["string"],
  "subcategories": ["string"],
  "specializations": ["string"],
  "keywords": ["string"]
}

# EXAMPLES
---
user_query: "서울 사는 25세 미취업자인데, 창업 지원금 좀 알아봐줘"
{
  "age": 25,
  "income": null,
  "regions": ["서울특별시"],
  "job_status": ["미취업자", "(예비)창업자"],
  "marriage_status": null,
  "education_levels": [],
  "majors": [],
  "categories": ["일자리"],
  "subcategories": ["창업"],
  "specializations": [],
  "keywords": ["보조금", "벤처"]
}
---
user_query: "강원 춘천에 거주하는 고졸 학력으로 지원 가능한 주거 대출 정책 있어?"
{
  "age": null,
  "income": null,
  "regions": ["강원특별자치도", "춘천시"],
  "job_status": [],
  "marriage_status": null,
  "education_levels": ["고교 졸업"],
  "majors": [],
  "categories": ["주거"],
  "subcategories": ["주택 및 거주지", "기숙사", "전월세 및 주거급여 지원"],
  "specializations": [],
  "keywords": ["대출"]
}
---
user_query: "목포에 사는 사람인데 석사 지원 정책같은거 있냐"
{
  "age": null,
  "income": null,
  "regions": ["전라남도", "목포시"],
  "job_status": [],
  "marriage_status": null,
  "education_levels": ["석·박사"],
  "majors": [],
  "categories": [],
  "subcategories": [],
  "specializations": [],
  "keywords": []
}
"""

    try:
        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o", 
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ]
        )
        
        # 반환된 JSON 문자열을 파이썬 딕셔너리로 파싱
        result = json.loads(response.choices[0].message.content)
        return result

    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

In [2]:
from dotenv import load_dotenv
load_dotenv()

os.getenv('OPENAI_API_KEY')
client = OpenAI()

# 사용자 질문
sample_query = "화순 사는 쌉백수인데, 관련 정책 좀 알아봐줘"

# 함수 호출
extracted_filters = create_filter_from_query(client, sample_query)

# 결과 출력
import pprint
pprint.pprint(extracted_filters)

{'age': None,
 'categories': [],
 'education_levels': [],
 'income': None,
 'job_status': ['미취업자'],
 'keywords': [],
 'majors': [],
 'marriage_status': None,
 'regions': ['전라남도', '화순군'],
 'specializations': [],
 'subcategories': []}


In [3]:
import mysql.connector
from mysql.connector import Error

def _get_all_related_region_codes(cursor, region_names: list) -> list:
    """
    지역명 리스트를 받아 관련된 모든 지역 코드(시/도 및 하위 시/군/구)를 반환합니다.
    """
    if not region_names:
        return []
    all_codes = set()
    region_regex = '|'.join(region_names)
    
    query = "SELECT code FROM region_codes WHERE sido REGEXP %s OR sigungu REGEXP %s"
    cursor.execute(query, (region_regex, region_regex))
    results = cursor.fetchall()
    for row in results:
        all_codes.add(row[0])
    return list(all_codes)


def get_rdb_candidate_ids(db_connection, filters: dict) -> list:
    """
    [최소 조건 버전] 기간과 지역 필터만을 사용하여 RDB에서 1차 후보군을 조회합니다.
    """
    try:
        cursor = db_connection.cursor()

        base_query = "SELECT DISTINCT p.policy_id FROM policies p"
        joins = set()
        where_conditions = []
        params = []

        # 1. 신청 기간 필터
        application_date_condition = """
        (p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))
        """
        where_conditions.append(application_date_condition.strip())
        
        # 2. 사업 기간 필터
        biz_date_condition = "(p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))"
        where_conditions.append(biz_date_condition)

        # 3. 지역 필터
        if filters.get("regions"):
            region_codes = _get_all_related_region_codes(cursor, filters["regions"])
            if region_codes:
                joins.add("JOIN policy_regions pr ON p.policy_id = pr.policy_id")
                placeholders = ', '.join(['%s'] * len(region_codes))
                where_conditions.append(f"pr.region_code IN ({placeholders})")
                params.extend(region_codes)
        
        # 최종 쿼리 조립
        final_query = base_query
        if joins:
            final_query += " " + " ".join(list(joins))
        if where_conditions:
            final_query += " WHERE (" + ") AND (".join(where_conditions) + ")"
        final_query += ";"

        # 디버깅을 위해 쿼리 템플릿과 파라미터를 별도로 출력
        print("--- Generated SQL Query (Simplified) ---")
        print(final_query)
        print("\n--- SQL Parameters ---")
        print(params)

        # 쿼리 실행
        cursor.execute(final_query, params)
        candidate_ids = [item[0] for item in cursor.fetchall()]
        return candidate_ids

    except Error as e:
        print(f"Database error: {e}")
        return []
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()

In [33]:
## 논리적 문제 : 해결 시도 버전

import mysql.connector
from mysql.connector import Error

def _get_all_related_region_codes(cursor, region_names: list) -> list:
    """
    지역명 리스트를 받아 관련된 모든 지역 코드(시/도 및 하위 시/군/구)를 반환합니다.
    """
    if not region_names:
        return []
    all_codes = set()
    region_regex = '|'.join(region_names)
    
    query = "SELECT code FROM region_codes WHERE sido REGEXP %s OR sigungu REGEXP %s"
    cursor.execute(query, (region_regex, region_regex))
    results = cursor.fetchall()
    for row in results:
        all_codes.add(row[0])
    return list(all_codes)


def get_rdb_candidate_ids(db_connection, filters: dict) -> list:
    """
    [시행 지역 기준] 최소 조건(기간, 시행 지역)만으로 RDB에서 1차 후보군을 조회합니다.
    """
    try:
        cursor = db_connection.cursor()

        base_query = "SELECT DISTINCT p.policy_id FROM policies p"
        where_conditions = []
        params = []

        # 1. 신청 기간 필터
        application_date_condition = """
        (p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))
        """
        where_conditions.append(application_date_condition.strip())
        
        # 2. 사업 기간 필터
        biz_date_condition = "(p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))"
        where_conditions.append(biz_date_condition)

        # 3. 시행 지역 필터 (수정된 로직)
        # 운영기관명 또는 감독기관명에 지역명이 포함되는지 검색
        if filters.get("regions"):
            region_like_conditions = []
            for region_name in filters["regions"]:
                # 예: (p.operInstCdNm LIKE '%서울%' OR p.sprvsnInstCdNm LIKE '%서울%')
                region_like_conditions.append("(p.operInstCdNm LIKE %s OR p.sprvsnInstCdNm LIKE %s)")
                # LIKE 검색을 위해 파라미터에 % 와일드카드 추가
                params.extend([f"%{region_name}%", f"%{region_name}%"])
            
            if region_like_conditions:
                # "서울" 또는 "종로구" 등 여러 지역명이 있을 경우 OR로 연결
                where_conditions.append("(" + " OR ".join(region_like_conditions) + ")")

        # 최종 쿼리 조립
        final_query = base_query
        if where_conditions:
            # 각 조건을 괄호로 묶어 AND로 연결
            final_query += " WHERE (" + ") AND (".join(where_conditions) + ")"
        final_query += ";"

        # 디버깅을 위해 쿼리 템플릿과 파라미터를 별도로 출력
        print("--- Generated SQL Query (Implementation Region Version) ---")
        print(final_query)
        print("\n--- SQL Parameters ---")
        print(params)

        # 쿼리 실행
        cursor.execute(final_query, params)
        candidate_ids = [item[0] for item in cursor.fetchall()]
        return candidate_ids

    except Error as e:
        print(f"Database error: {e}")
        return []
    finally:
        if 'cursor' in locals() and cursor:
            cursor.close()

In [34]:
def create_db_connection(host_name, user_name, user_password, db_name):
    """
    MySQL 데이터베이스 연결을 생성합니다.
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        if connection.is_connected():
            db_info = connection.get_server_info()
            print(f"MySQL 서버 버전: {db_info}")
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print(f"연결된 데이터베이스: {record}")
            return connection
    except Error as e:
        print(f"데이터베이스 연결 오류 발생: {e}")
    return connection

In [35]:
conn = create_db_connection("localhost", "root", "1234", "toyprj4")

MySQL 서버 버전: 8.0.37-0ubuntu0.20.04.3
연결된 데이터베이스: ('toyprj4',)


/tmp/ipykernel_4703/1286675617.py:14: DeprecationWarning: Call to deprecated function get_server_info. Reason: 
    The property counterpart 'server_info' should be used instead.

  db_info = connection.get_server_info()


In [38]:
d = get_rdb_candidate_ids(conn, extracted_filters)
pprint.pprint(d)

# %s : SQL 인젝션 방지 목적의 플레이스 홀더.

--- Generated SQL Query (Implementation Region Version) ---
SELECT DISTINCT p.policy_id FROM policies p WHERE ((p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))) AND ((p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))) AND (((p.operInstCdNm LIKE %s OR p.sprvsnInstCdNm LIKE %s) OR (p.operInstCdNm LIKE %s OR p.sprvsnInstCdNm LIKE %s)));

--- SQL Parameters ---
['%전라남도%', '%전라남도%', '%화순군%', '%화순군%']
Database error: 1054 (42S22): Unknown column 'p.operInstCdNm' in 'where clause'
[]


In [36]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

def semantic_search(
    candidate_ids: list,
    original_query: str,
    extracted_filters: dict,
    vdb_directory: str,
    k: int = 5,
    fetch_k: int = 20,
    lambda_mult: float = 0.7
) -> list:
    '''
    전달받은 필터를 통해 사용자의 질문을 증강하고, 후보 인덱스 내에서만 R을 수행하여 검색 정확도를 높여 시멘틱 서칭을 수행하는 함수입니다.
    Args:
        candidate_ids (list): 검색할 후보 ID 목록입니다.
        original_query (str): 원본 검색 쿼리 문자열입니다.
        extracted_filters (dict): 추출된 필터 정보 딕셔너리입니다.
        vdb_directory (str): 벡터 데이터베이스 디렉토리 경로입니다.
        k (int, optional): 반환할 상위 검색 결과의 개수입니다. 기본값은 5입니다.
        fetch_k (int, optional): 유사도 검색을 위해 가져올 초기 결과의 개수입니다. 기본값은 20입니다.
        lambda_mult (float, optional): 재정렬(reranking) 시 사용되는 람다 값입니다. 기본값은 0.7입니다.
    '''
    if not candidate_ids:
        return []

    # 1. 보강된 검색어 생성
    boost_keywords = []
    soft_filter_keys = ["job_status", "education_levels", "keywords", "categories"]
    for key in soft_filter_keys:
        if extracted_filters.get(key):
            boost_keywords.extend(extracted_filters[key])
    
    synthetic_query = original_query + " " + " ".join(list(set(boost_keywords)))
    print(f"\n--- Generated Vector Search Query ---\n{synthetic_query}\n")

    # 2. 임베딩 모델 및 ChromaDB 로드
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = Chroma(
        collection_name="policy_collection_summary_added_openai_large", 
        
        embedding_function=embedding_model,
        persist_directory=vdb_directory
    )

    # 3. 필터가 적용된 Retriever 생성
    retriever = vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": k,
            "fetch_k": fetch_k,
            "lambda_mult": lambda_mult,
            "filter": {'plcyNo': {'$in': candidate_ids}}
        }
    )

    # 4. Retriever 실행 및 Document 리스트 반환
    docs = retriever.invoke(synthetic_query)
    
    return docs

In [37]:
docs = semantic_search(d,sample_query,extracted_filters,'chroma_db_policy')
# docs = semantic_search(d,sample_query,extracted_filters,'chroma_db_policy_chunk_kss_model_tiny/')
print(docs)


--- Generated Vector Search Query ---
화순 사는 쌉백수인데, 관련 정책 좀 알아봐줘 미취업자

[Document(metadata={'plcyAplyMthdCn': '○ 방문 신청 또는 우편 : 고용센터\n○ 온라인 신청 : 고용24 홈페이지(www.work24.go.kr)\n○ 지급 결정 / 지급 : 신청 접수 후 14일 이내 지급(부지급) 결정 및 통지 / 본인 계좌로 지급 => 별도로 신청절차를 넣어서 표기', 'bizPrdEtcCn': '', 'sprvsnInstCdNm': '고용노동부', 'aplyYmd': '', 'plcyMajorCd': '0011009', 'addAplyQlfcCndCn': "○ 출산 전 18개월중 3개월 이상 소득활동이 확인되어야 합니다.\n\n○ 근로자\n- 1유형 : 출산전 30일 이상 고용보험에 가입하고 있고, 고용보험의 출산전후휴가급여 수급요건을 미충족*한 근로자\n· 출산전후휴가가 끝난 날 전까지 피보험 단위 기간이 통산하여 180일 이상\n- 2유형 : 고용보험법상 고용보험 적용제외 사업*의 근로자 및 적용제외 근로자**\n* 적용제외 사업: 농,임,어업 중 법인이 아닌 자가 상시 4명 이하의 근로자를 사용하는 사업, 총 공사금액이 2천만원 미만인 공사, 연면적 100㎡ 이하 건축물의 건축, 200㎡ 이하 건축물의 대수선 공사\n- 적용제외 사업의 입증 : 관련 업종이 명시된 사업자등록증 또는 공적 기관이 발급한 확인서(예시:농업경영체등록확인서, 축산업허가증, 어업 경영체등록확인서, 어업허가증, 임업 후계자 증서, 공사금액이 명시된 도급계약서 등)\n** 적용제외 근로자 : 소정 근로시간이 월 60시간 미만인 근로자(주 15시간 미만 포함)\n- 3유형 : 고용보험 미성립 사업장 소속의 고용보험 미가입 근로자(고용보험 가입 의무가 있는 사업장이지만 성립신고를 하지 않아, 소속 근로자 또한 피보험자격을 갖지 못한 경우)\n\n○ 1인 사업자\n- 출산일 현재 피고용인이 없는 단독ㆍ공동사업자

In [39]:
# 필요한 모든 함수와 라이브러리를 임포트했다고 가정합니다.
# from step1_parser import create_filter_from_query
# from step2_rdb_filter import get_rdb_candidate_ids
# from step3_semantic_search import semantic_search
from openai import OpenAI
import mysql.connector

# --- 0. 기본 설정 ---
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
VDB_DIRECTORY = "./chroma_db_policy"
openai_client = OpenAI()

# --- 1. 사용자 질문 ---
user_query = "구로구에 사는데 주거지원 정책좀 알려줘."
print(f"--- User Query ---\n{user_query}\n")


# --- 2. 1단계: LLM 파서 실행 ---
llm_filters = create_filter_from_query(openai_client, user_query)
print(f"--- Step 1: LLM Filter Results ---\n{llm_filters}\n")


# --- 3. 2단계: RDB 필터링 실행 ---
try:
    connection = mysql.connector.connect(
        host='localhost',
        database='toyprj4',
        user='root',
        password='1234'
    )
    rdb_candidate_ids = get_rdb_candidate_ids(connection, llm_filters)
    print(f"--- Step 2: RDB Candidate IDs ---\n{rdb_candidate_ids}\n")

finally:
    if 'connection' in locals() and connection.is_connected():
        connection.close()


# --- 4. 3단계: Vector DB 랭킹 실행 ---
final_documents = semantic_search(
    candidate_ids=rdb_candidate_ids,
    original_query=user_query,
    extracted_filters=llm_filters,
    vdb_directory=VDB_DIRECTORY
)


# --- 5. 최종 결과 확인 ---
print("\n--- Step 3: Final Retrieved Documents ---")
for doc in final_documents:
    print(f"Policy ID: {doc.metadata.get('plcyNo')}")
    print(f"application date: {doc.metadata.get('aplyYmd')}")
    print(f"Content: {doc.page_content}...") # 내용 일부만 출력
    print("-" * 20)

--- User Query ---
구로구에 사는데 주거지원 정책좀 알려줘.

--- Step 1: LLM Filter Results ---
{'age': None, 'income': None, 'regions': ['구로구'], 'job_status': [], 'marriage_status': None, 'education_levels': [], 'majors': [], 'categories': ['주거'], 'subcategories': ['주택 및 거주지', '기숙사', '전월세 및 주거급여 지원'], 'specializations': [], 'keywords': ['주거지원']}

--- Generated SQL Query (Implementation Region Version) ---
SELECT DISTINCT p.policy_id FROM policies p WHERE ((p.application_status = '상시' OR 
         (p.application_status = '특정 기간' AND CURDATE() BETWEEN DATE(p.aply_start_date) AND DATE(p.aply_end_date)))) AND ((p.biz_end_date IS NULL OR CURDATE() <= DATE(p.biz_end_date))) AND (((p.operInstCdNm LIKE %s OR p.sprvsnInstCdNm LIKE %s)));

--- SQL Parameters ---
['%구로구%', '%구로구%']
Database error: 1054 (42S22): Unknown column 'p.operInstCdNm' in 'where clause'
--- Step 2: RDB Candidate IDs ---
[]


--- Step 3: Final Retrieved Documents ---


In [ ]:
# 오케이 일단 확인 완료

In [17]:
import chromadb

# 본인의 Vector DB 경로를 정확하게 입력해주세요.
VDB_DIRECTORY = "./chroma_db_policy/" 
# 인덱싱 시 사용한 컬렉션 이름을 정확하게 입력해주세요.
COLLECTION_NAME = "policy_collection_summary_added_openai_large" 

try:
    # 1. 저장된 DB 로드
    client = chromadb.PersistentClient(path=VDB_DIRECTORY)
    collection = client.get_collection(name=COLLECTION_NAME)

    # 2. 필터 없이 데이터 5개 가져오기
    results = collection.get(
        limit=1,
        include=["metadatas", "documents"] # 메타데이터와 문서 내용을 함께 확인
    )

    # 3. 결과 출력
    print("--- ChromaDB에 저장된 데이터 샘플 ---")
    import pprint
    pprint.pprint(results)

except Exception as e:
    print(f"오류가 발생했습니다: {e}")
    print("Vector DB 경로와 컬렉션 이름이 올바른지 다시 확인해주세요.")

--- ChromaDB에 저장된 데이터 샘플 ---
{'data': None,
 'documents': ["정책명은 '웹툰캠퍼스 운영 및 인력 양성'입니다. 정책 분야는 '복지문화 > 예술인지원'입니다. 주요 키워드는 "
               "'교육지원, 인턴, 맞춤형상담서비스, 벤처'입니다. 정책 설명: - 현장 실무형 전문인력 프로그램을 통한 글로벌 "
               '웹툰작가 양성\n'
               '- 웹툰 창작 및 제작 역량 강화, 산업 활성화 지원 지원 내용: - 지역 웹툰작가 양성을 위한 교육 및 '
               '프로그램 운영\n'
               '- 교육생 월 창작지원금 및 원고료 지원 등'],
 'embeddings': None,
 'ids': ['5f71d393-54c7-4ae0-9126-f60dc40301f7'],
 'included': ['metadatas', 'documents'],
 'metadatas': [{'addAplyQlfcCndCn': '',
                'aplyPrdSeCd': '57001',
                'aplyUrlAddr': '',
                'aplyYmd': '20250501 ~ 20250530',
                'bizPrdBgngYmd': '20250501',
                'bizPrdEndYmd': '20251231',
                'bizPrdEtcCn': '',
                'bizPrdSeCd': '56001.0',
                'earnEtcCn': '',
                'earnMaxAmt': '0.0',
                'earnMinAmt': '0.0',
                'etcMttrCn': '',
                'frstRegDt': '2025-07-